In [1]:
import pandas as pd
import numpy as np
import re
import os

import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.stem.isri import ISRIStemmer

!pip install Tashaphyne
import pyarabic.arabrepr
from tashaphyne.stemming import ArabicLightStemmer

import random
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv('D:/NLP/arabic data set/archive/ar_reviews_3.tsv', sep='\t')
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [3]:
stop_words =['من',
 'في',
 'على',
 'و',
 'فى',
 'يا',
 'عن',
 'مع',
 'ان',
 'هو',
 'علي',
 'ما',
 'اللي',
 'كل',
 'بعد',
 'ده',
 'اليوم',
 'أن',
 'يوم',
 'انا',
 'إلى',
 'كان',
 'ايه',
 'اللى',
 'الى',
 'دي',
 'بين',
 'انت',
 'أنا',
 'حتى',
 'لما',
 'فيه',
 'هذا',
 'واحد',
 'احنا',
 'اي',
 'كده',
 'إن',
 'او',
 'أو',
 'عليه',
 'ف',
 'دى',
 'مين',
 'الي',
 'كانت',
 'أمام',
 'زي',
 'يكون',
 'خلال',
 'ع',
 'كنت',
 'هي',
 'فيها',
 'عند',
 'التي',
 'الذي',
 'قال',
 'هذه',
 'قد',
 'انه',
 'ريتويت',
 'بعض',
 'أول',
 'ايه',
 'الان',
 'أي',
 'منذ',
 'عليها',
 'له',
 'ال',
 'تم',
 'ب',
 'دة',
 'عليك',
 'اى',
 'كلها',
 'اللتى',
 'هى',
 'دا',
 'انك',
 'وهو',
 'ومن',
 'منك',
 'نحن',
 'زى',
 'أنت',
 'انهم',
 'معانا',
 'حتي',
 'وانا',
 'عنه',
 'إلي',
 'ونحن',
 'وانت',
 'منكم',
 'وان',
 'معاهم',
 'معايا',
 'وأنا',
 'عنها',
 'إنه',
 'اني',
 'معك',
 'اننا',
 'فيهم',
 'د',
 'انتا',
 'عنك',
 'وهى',
 'معا',
 'آن',
 'انتي',
 'وأنت',
 'وإن',
 'ومع',
 'وعن',
 'معاكم',
 'معاكو',
 'معاها',
 'وعليه',
 'وانتم',
 'وانتي',
 '¿',
 '|']

In [4]:
def normalize(sentence):
    '''
    Argument:
        string of words
    return:
        string of words but standardize the words
    '''
    sentence = re.sub("[إأآا]", "ا", sentence)
    sentence = re.sub("ى", "ي", sentence)
    sentence = re.sub("ؤ", "ء", sentence)
    sentence = re.sub("ئ", "ء", sentence)
    sentence = re.sub("ة", "ه", sentence)
    sentence = re.sub("گ", "ك", sentence)
    return sentence

In [5]:
def removing_ar_stopwords(text):
    """
        Here we remove all Arabic stop words
        
    """
      # if read it from file
#     ar_stopwords_list = open("arabic_stopwords.txt", "r") 
#     stop_words = ar_stopwords_list.read().split("\n")
#     stop_words = []
    original_words = []
    words = word_tokenize(text) # it works on one sentence not list
    for word in words:
        if word not in stop_words:
            original_words.append(word)
    filtered_sentence = " ".join(original_words)
    return filtered_sentence

In [6]:
def clearReg(text):
    """
        This function for getting the normal values of out of lemmatization function
        that takse a string of dict as a 
        takes  : '{"result":["امر","ب","أخذ","ما","نهى","ه","انتهى"]}'
        return : ['امر أخذ ما نهى انتهى']
    """
    each_lemma_word = []
    each_lemma_sentence = []
    for final_data in text:
        matches = re.findall(r'\"(.+?)\"',final_data)
        for word in matches:
            if len(word) >= 2 and word !='result':
                each_lemma_word.append(word)
        each_lemma_sentence.append(" ".join(each_lemma_word))
        each_lemma_word.clear()
    return each_lemma_sentence

In [7]:
#stemming_2 by Tashaphyne is an Arabic light stemmer(removing prefixes and suffixes) and give all possible segmentations.
#Stemming : algorithms work by cutting off the end or the beginning of the word, taking into account a list of common prefixes and suffixes that can be found in an inflected word.

def stemming_2(text):
    """
        This is  functoin for stemming and it's looks good results, with built in library called Tashaphyne.
        The documentation here ==> https://pypi.org/project/Tashaphyne/
    
    """
    import pyarabic.arabrepr
    arepr = pyarabic.arabrepr.ArabicRepr()
    repr = arepr.repr

    from tashaphyne.stemming import ArabicLightStemmer
    ArListem = ArabicLightStemmer()

    final_data_without_stop_words_and_with_normalization_and_with_stemming = []

    for final_data in final_data_without_stop_words_and_with_normalization:
        words = word_tokenize(final_data)
        new_list = []
        for word in words:
            stem = ArListem.light_stem(word)
            stem = ArListem.get_stem()
            new_list.append(stem)

        final_data_sentence_with_stemming = " ".join(new_list)
        final_data_without_stop_words_and_with_normalization_and_with_stemming.append(final_data_sentence_with_stemming)
        
    return final_data_without_stop_words_and_with_normalization_and_with_stemming

In [8]:
#lemmatization by Farasa API
#Lemmatization : takes into consideration the morphological analysis of the words.

def lemmatization(text):
    """
        This function for lemma Arabic words by API, and it getting best result of the previous functions
        return a string dictinary like exactly '{"result":["امر","ب","أخذ","ما","نهى","ه","انتهى"]}'
    """
    import http.client
    conn = http.client.HTTPSConnection("farasa-api.qcri.org")
    final_data_dict = {}
    list_pyload_input = []
    list_pyload_out = []
    length = len(text)
    for h in text[:length]:
        q = '{"text":'+'"{}"'.format(h)+'}'
        list_pyload_input.append(q)
    headers = { 'content-type': "application/json", 'cache-control': "no-cache", }
    for h in list_pyload_input:
        conn.request("POST", "/msa/webapi/lemma", h.encode('utf-8'), headers)
        res = conn.getresponse()
        data = res.read()
        list_pyload_out.append(data.decode("utf-8"))
        final_result = clearReg(list_pyload_out)     # call clearReg for clean the text
    return final_result

In [9]:
data= []

for words in df['text']:
    data.append(words)

In [10]:
%%time

# sentiment analysis first step which is cleanig data
cleared_data_1 = []           # Removing stopwords
cleared_data_1_2 = []         # Normalization
#cleared_data_1_2_3 = []       # Lematization

for final_data in data:
    cleared_data_1.append(removing_ar_stopwords(final_data))         # Removing stopwords
len(cleared_data_1)

Wall time: 41.7 s


59999

In [11]:
for final_data in cleared_data_1:
    cleared_data_1_2.append(normalize(final_data))                   # Normalization
    
len(cleared_data_1_2)

59999

In [ ]:
#cleared_data_1_2_3 = lemmatization(cleared_data_1_2)           # Lematization

#print('The size of data:')
#len(cleared_data_1), len(cleared_data_1_2), len(cleared_data_1_2_3)

In [12]:
df['clean_data'] = cleared_data_1_2



In [13]:
df.to_csv('D:/NLP/arabic data set/archive/file.tsv',sep="\t",encoding='utf-8-sig')


In [14]:
df.head()

,label,text,clean_data
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...,ممتاز نوعا . النظافه والموقع والتجهيز والشاطيء...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...,احد اسباب نجاح الامارات شخص الدوله يعشق ترابها...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...,هادفه .. وقويه . تنقلك صخب شوارع القاهره هدوء ...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...,خلصنا .. مبدءيا مستني ابهار الفيل الازرق ميقرا...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...,ياسات جلوريا جزء لا يتجزا دبي . فندق متكامل ال...


In [15]:
X = df['clean_data']
y = df['label']

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print('Training Data Shape:', X_train.shape)
print('Testing Data Shape: ', X_test.shape)

Training Data Shape: (40199,)
Testing Data Shape:  (19800,)


In [17]:
y_train.value_counts()

Mixed       13425
Positive    13399
Negative    13375
Name: label, dtype: int64

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(40199, 164029)

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)

In [20]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[3470 1447 1658]
 [1296 4694  635]
 [1308  648 4644]]


In [21]:
print(metrics.classification_report(y_test,predictions)) 

              precision    recall  f1-score   support

       Mixed       0.57      0.53      0.55      6575
    Negative       0.69      0.71      0.70      6625
    Positive       0.67      0.70      0.69      6600

    accuracy                           0.65     19800
   macro avg       0.64      0.65      0.64     19800
weighted avg       0.64      0.65      0.65     19800

